# Complete MoS2 Analysis Pipeline - All 3 Stages

## Based on your feedback, this notebook:
- **Stage 1**: Uses threshold < 140 (which worked well for your images)
- **Stage 2**: Detects multilayer structures within flakes  
- **Stage 3**: Calculates twist angles between layers

## Fixed Issues:
- ✅ Matplotlib subplot error fixed
- ✅ Optimized for threshold 140 detection
- ✅ Added noise filtering for better results
- ✅ Complete 3-stage pipeline

In [ ]:
# Install required packages
!pip install opencv-python-headless matplotlib numpy scipy scikit-image

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
from scipy import ndimage
from skimage import measure, morphology, filters
import os
from scipy.spatial.distance import cdist

# Create directories
os.makedirs('/content/images', exist_ok=True)
os.makedirs('/content/results', exist_ok=True)
os.makedirs('/content/debug', exist_ok=True)

print("✓ Environment setup complete")

In [ ]:
    def find_internal_structures(self, roi_gray, roi_mask, offset_x, offset_y):
        """Find internal structures within a flake ROI"""
        internal_structures = []
        
        # Method 1: Edge detection within the flake
        edges = cv2.Canny(roi_gray, 30, 90)
        edges = cv2.bitwise_and(edges, roi_mask)
        
        # Dilate edges slightly to connect gaps
        kernel = np.ones((2,2), np.uint8)
        edges = cv2.dilate(edges, kernel, iterations=1)
        
        # Find internal contours
        contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            area = cv2.contourArea(contour)
            
            # Check if this internal structure is significant
            roi_area = np.sum(roi_mask > 0)  # Fixed: count non-zero pixels properly
            area_ratio = area / roi_area if roi_area > 0 else 0
            
            if area > 100 and area_ratio > 0.05:  # At least 5% of the flake area
                # Adjust coordinates back to full image
                adjusted_contour = contour + [offset_x, offset_y]
                
                # Check if it's roughly triangular
                epsilon = 0.03 * cv2.arcLength(contour, True)
                approx = cv2.approxPolyDP(contour, epsilon, True)
                
                if 3 <= len(approx) <= 7:  # Roughly polygonal
                    internal_structures.append({
                        'contour': adjusted_contour,
                        'approx': approx + [offset_x, offset_y],
                        'area': area,
                        'vertices': len(approx)
                    })
        
        # Method 2: Intensity-based detection of darker regions within flake
        masked_roi = cv2.bitwise_and(roi_gray, roi_mask)
        if masked_roi.max() > 0:
            # Find darker regions within the flake
            mask_pixels = masked_roi[roi_mask > 0]
            if len(mask_pixels) > 0:  # Check if we have valid pixels
                mean_intensity = mask_pixels.mean()
                dark_threshold = mean_intensity - 10  # Slightly darker than average
                
                dark_regions = (masked_roi < dark_threshold) & (roi_mask > 0)
                dark_regions = dark_regions.astype(np.uint8) * 255
                
                # Clean up dark regions
                kernel = np.ones((3,3), np.uint8)
                dark_regions = cv2.morphologyEx(dark_regions, cv2.MORPH_OPEN, kernel)
                
                # Find contours of dark regions
                dark_contours, _ = cv2.findContours(dark_regions, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                for contour in dark_contours:
                    area = cv2.contourArea(contour)
                    if area > 150:  # Minimum size for internal structure
                        # Check if not already found by edge method
                        adjusted_contour = contour + [offset_x, offset_y]
                        
                        # Simple duplicate check
                        is_duplicate = False
                        for existing in internal_structures:
                            if abs(existing['area'] - area) < 50:  # Similar area
                                is_duplicate = True
                                break
                        
                        if not is_duplicate:
                            epsilon = 0.03 * cv2.arcLength(contour, True)
                            approx = cv2.approxPolyDP(contour, epsilon, True)
                            
                            internal_structures.append({
                                'contour': adjusted_contour,
                                'approx': approx + [offset_x, offset_y],
                                'area': area,
                                'vertices': len(approx),
                                'detection_method': 'intensity'
                            })
        
        return internal_structures

In [ ]:
# Run complete pipeline on all images
image_dir = Path('/content/images')
image_extensions = ['.jpg', '.jpeg', '.png', '.tiff', '.bmp']

image_files = []
for ext in image_extensions:
    image_files.extend(image_dir.glob(f'*{ext}'))
    image_files.extend(image_dir.glob(f'*{ext.upper()}'))

print(f"Found {len(image_files)} images to process\n")

all_results = []

for image_path in image_files:
    try:
        # Run complete pipeline
        results = pipeline.process_complete_pipeline(str(image_path))
        
        # Save visualization
        plt.figure(results['figure'].number)
        plt.savefig(f'/content/results/{image_path.stem}_complete_analysis.png', 
                   dpi=300, bbox_inches='tight')
        plt.show()
        
        # Print detailed results
        print(f"\n{'='*40}")
        print(f"DETAILED RESULTS")
        print(f"{'='*40}")
        
        flakes = results['flakes']
        multilayer_flakes = results['multilayer_flakes']
        angle_results = results['angle_results']
        
        print(f"\nFLAKE SUMMARY:")
        for flake in flakes:
            layer_info = f" ({flake['layer_count']}L)" if flake.get('is_multilayer') else " (1L)"
            print(f"  Flake {flake['id']}: Area={flake['area']:.0f}px, "
                  f"Vertices={flake['vertices']}, Solidity={flake['solidity']:.2f}{layer_info}")
        
        if angle_results:
            print(f"\nTWIST ANGLES:")
            for result in angle_results:
                print(f"  Flake {result['flake_id']}: {result['average_twist']:.1f}° "
                      f"(from {len(result['twist_measurements'])} measurements)")
                for i, measurement in enumerate(result['twist_measurements']):
                    print(f"    Layer {i+1}: {measurement['twist_angle']:.1f}° "
                          f"(area: {measurement['internal_area']:.0f}px)")
        
        # Save detailed JSON results
        json_data = {
            'filename': image_path.name,
            'analysis_summary': {
                'total_flakes': len(flakes),
                'multilayer_flakes': len(multilayer_flakes),
                'bilayer_structures': len(angle_results)
            },
            'flake_details': [],
            'twist_angle_data': []
        }
        
        # Save flake details
        for flake in flakes:
            flake_data = {
                'id': flake['id'],
                'area': float(flake['area']),
                'vertices': int(flake['vertices']),
                'solidity': float(flake['solidity']),
                'aspect_ratio': float(flake['aspect_ratio']),
                'is_multilayer': bool(flake.get('is_multilayer', False)),
                'layer_count': int(flake.get('layer_count', 1)),
                'centroid': flake['centroid']
            }
            json_data['flake_details'].append(flake_data)
        
        # Save twist angle data
        for result in angle_results:
            angle_data = {
                'flake_id': result['flake_id'],
                'main_angle': float(result['main_angle']),
                'average_twist': float(result['average_twist']),
                'individual_measurements': [
                    {
                        'twist_angle': float(m['twist_angle']),
                        'internal_area': float(m['internal_area'])
                    } for m in result['twist_measurements']
                ]
            }
            json_data['twist_angle_data'].append(angle_data)
        
        # Save to file
        with open(f'/content/results/{image_path.stem}_complete_results.json', 'w') as f:
            json.dump(json_data, f, indent=2)
        
        all_results.append(json_data)
        print(f"\n✓ Complete results saved for {image_path.name}")
        
    except Exception as e:
        print(f"❌ Error processing {image_path.name}: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

# Final summary
if all_results:
    print(f"\n{'='*60}")
    print(f"FINAL SUMMARY - ALL IMAGES")
    print(f"{'='*60}")
    
    total_flakes = sum(r['analysis_summary']['total_flakes'] for r in all_results)
    total_multilayer = sum(r['analysis_summary']['multilayer_flakes'] for r in all_results)
    total_bilayer = sum(r['analysis_summary']['bilayer_structures'] for r in all_results)
    
    print(f"Images processed: {len(all_results)}")
    print(f"Total flakes detected: {total_flakes}")
    print(f"Total multilayer structures: {total_multilayer}")
    print(f"Total bilayer structures with twist angles: {total_bilayer}")
    
    if total_flakes > 0:
        print(f"Multilayer detection rate: {total_multilayer/total_flakes*100:.1f}%")
    
    # Collect all twist angles
    all_twist_angles = []
    for result in all_results:
        for angle_data in result['twist_angle_data']:
            all_twist_angles.append(angle_data['average_twist'])
    
    if all_twist_angles:
        print(f"\nTwist angle statistics:")
        print(f"  Total measurements: {len(all_twist_angles)}")
        print(f"  Mean: {np.mean(all_twist_angles):.1f}°")
        print(f"  Median: {np.median(all_twist_angles):.1f}°")
        print(f"  Range: {np.min(all_twist_angles):.1f}° - {np.max(all_twist_angles):.1f}°")
        print(f"  Standard deviation: {np.std(all_twist_angles):.1f}°")
    
    # Save overall summary
    with open('/content/results/complete_pipeline_summary.json', 'w') as f:
        json.dump(all_results, f, indent=2)
    
    print(f"\n✓ Complete pipeline results saved to /content/results/")
else:
    print("No images were successfully processed.")

## Results Interpretation Guide

### Output Files:
- **`[filename]_complete_analysis.png`**: Visual summary of all 3 stages
- **`[filename]_complete_results.json`**: Detailed numerical data
- **`complete_pipeline_summary.json`**: Summary of all processed images

### Stage Results:
1. **Stage 1 (Blue outlines)**: All detected MoS2 flakes
2. **Stage 2 (Green/Cyan outlines)**: Multilayer structures with internal features
3. **Stage 3 (Orange outlines + angles)**: Bilayer twist angles in degrees

### Key Metrics:
- **Layer count**: Number of layers detected (1L, 2L, 3L, etc.)
- **Twist angles**: Rotation between overlapping triangular layers
- **Detection rate**: Percentage of flakes showing multilayer structure

### Quality Indicators:
- **Good detection**: Clear triangular outlines, reasonable twist angles (0-60°)
- **False positives**: Very small areas, irregular shapes, extreme angles
- **Missed structures**: Manual inspection may reveal additional multilayer regions

### Next Steps:
If results need refinement, adjust these parameters in the code:
- `intensity_threshold` (currently 140)
- `min_flake_area` (currently 200)
- Internal structure detection sensitivity
